# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл


In [1]:
import pandas as pd# Импортирую нужные в работе библиотеки.

In [2]:
from sklearn.tree import DecisionTreeClassifier

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.linear_model import LogisticRegression 

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
try:

    df = pd.read_csv('C:/your_project/users_behavior.csv')
except:

    df = pd.read_csv('/datasets/users_behavior.csv')

In [8]:
df.info()# Изучим общую информацию о полученном датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [9]:
df.head(10)# взглянем на первые 10 строчек 

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## Разбейте данные на выборки

In [10]:
df_tr, df_valid = train_test_split(df, test_size=0.2, stratify=df['is_ultra'], random_state=12345) # отделите 20% данных для валидационной выборки

In [11]:
df_train, df_test = train_test_split(df_tr, test_size=0.25, stratify=df_tr['is_ultra'], random_state=12345) # отделите 20% данных для тестовой выборки

## Исследуйте модели

Создадим обучающий набор данных

In [12]:
features = df_train.drop(['is_ultra'], axis=1)
target = df_train['is_ultra']

In [13]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [14]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Начнем с модели 'Дерево решений'

In [15]:
best_model_dtc= None
best_result_dtc= 0

for depth in range(1, 50):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)# применим метод
    model_dtc.fit(features, target)# обучим модель
    predictions_valid = model_dtc.predict(features_valid)
    result_dtc = accuracy_score(target_valid, predictions_valid)# качество модели посчитаем на валидационной выборке
    if result_dtc > best_result_dtc:
        best_model_dtc= model_dtc# сохраним наилучшую модель
        best_result_dtc= result_dtc#  сохраним наилучшее значение метрики accuracy на валидационных данных
        best_depth = depth

best_result_dtc, best_depth

(0.7993779160186625, 3)

Результат accuracy модели 'Дерево решений' равен 0.7993779160186625, а лучшая глубина дерева равна 3. Неплохо.

Далее перейдем к модели 'Логистическая регрессия'.

In [16]:
model_lr = LogisticRegression(random_state=12345)# применим метод
model_lr.fit(features, target)# обучим модель
result_lr = model_lr.score(features_valid, target_valid)# качество модели посчитаем на валидационной выборке
result_lr

0.7371695178849145

Результат accuracy модели 'Логистическая регрессия' равен 0.7371695178849145. Заметно хуже, чем в 'Дерево решений'.

Далее посмотрим на модель 'Случайный лес'.

In [17]:
best_model_rfc = None
best_result_rfc = 0


for est in range(1, 20):
    for depth in range (1, 20):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rfc.fit(features, target)
        result_rfc = model_rfc.score(features_valid, target_valid)
        
        if result_rfc > best_result_rfc:
            best_model_rfc = model_rfc
            best_result_rfc = result_rfc
            best_est_rfc = est
            best_depth_rfc = depth
            
best_result_rfc, best_depth_rfc, best_est_rfc

(0.8195956454121306, 17, 18)

Кажется у нас есть победитель. Результат accuracy модели 'Случайный лес' равен 0.8195956454121306.

## Проверьте модель на тестовой выборке

In [23]:
model = RandomForestClassifier(random_state=12345, n_estimators=17, max_depth=18)
model.fit(features, target)
result = model.score(features_test, target_test)
result     

0.8087091757387247

После проверки на тестовой выборке результат получился немного хуже, но совсем не критично!

## (бонус) Проверьте модели на адекватность

In [19]:
target.value_counts()# посмотрим большую долю ответов

0    1337
1     591
Name: is_ultra, dtype: int64

In [20]:
target.count()# посмотрим общее количество записей

1928

In [22]:
1337 / 1928# посчитаем долю случайных значений для сравнения с нашим accuracy

0.6934647302904564

Наша модель в сравнении со случайными показателями показывает более высокий результат.Признаем ее адекватной!)

Подведем итог.

Самой подходящей моделью для совета аббонентам нового тарифа стала 'RandomForestClassifier' с максимальным количеством - 17 деревьев и максимальной глубиной - 18!